In [ ]:
import pandas as pd
import networkx as nx
from collections import deque

# Create the DataFrame
data = {
    'sending application': [123, 456, 789, 666, 110],
    'upstream application': [456, 789, 110, 110, 879]
}

df = pd.DataFrame(data)

# Create the DiGraph from the DataFrame
G = nx.from_pandas_edgelist(df, 'sending application', 'upstream application', create_using=nx.DiGraph())

df


,sending application,upstream application
0,123,456
1,456,789
2,789,110
3,666,110
4,110,879


In [ ]:
import pandas as pd
import networkx as nx

# Create the DataFrame
data = {
    'sending application': [123, 456, 789, 666, 110],
    'upstream application': [456, 789, 110, 110, 879]
}
df = pd.DataFrame(data)

# Create the DiGraph from the DataFrame
G = nx.from_pandas_edgelist(df, 'sending application', 'upstream application', create_using=nx.DiGraph())

# DFS function to find all paths
def dfs_paths(G, start, is_goal, path=None):
    if path is None:
        path = [start]
    if is_goal(start):
        yield path
    for next_node in G.neighbors(start):
        if next_node not in path:  # Avoid cycles
            yield from dfs_paths(G, next_node, is_goal, path + [next_node])

# Find all paths for every starting node using DFS
all_paths_dfs = {}
for start in df['sending application']:
    all_paths_dfs[start] = list(dfs_paths(G, start, lambda node: True))

# Convert the results to a DataFrame
all_results = []
for start, paths in all_paths_dfs.items():
    for path in paths:
        all_results.append([path[0], path[-1], '->'.join(map(str, path))])

df_all_results = pd.DataFrame(all_results, columns=['Start Node', 'End Node', 'Path'])
df_all_results


,Start Node,End Node,Path
0,123,123,123
1,123,456,123->456
2,123,789,123->456->789
3,123,110,123->456->789->110
4,123,879,123->456->789->110->879
5,456,456,456
6,456,789,456->789
7,456,110,456->789->110
8,456,879,456->789->110->879
9,789,789,789


In [ ]:
def iddfs(G, source, max_depth=None):
    """Iterative Deepening Depth First Search."""
    def DLS(node, depth):
        if depth == 0:
            return [[node]]
        if depth > 0 and list(G.successors(node)):
            paths = []
            for successor in G.successors(node):
                for path in DLS(successor, depth - 1):
                    paths.append([node] + path)
            return paths
        return []

    if max_depth is None:
        max_depth = len(G)
    for depth in range(max_depth):
        paths = DLS(source, depth)
        if paths:
            yield from paths

# Find the longest path for every starting node using IDDFS
all_paths_iddfs = {}
MAX_DEPTH = None
for start in df['sending application']:
    paths = list(iddfs(G, start, MAX_DEPTH))
    if paths:
        # Storing only the longest path for each start node to save memory
        all_paths_iddfs[start] = max(paths, key=len)

# Convert the results to a DataFrame
results_iddfs = []
for start, path in all_paths_iddfs.items():
    results_iddfs.append([path[0], path[-1], '->'.join(map(str, path))])

df_results_iddfs = pd.DataFrame(results_iddfs, columns=['Start Node', 'End Node', 'Path'])
df_results_iddfs


,Start Node,End Node,Path
0,123,879,123->456->789->110->879
1,456,879,456->789->110->879
2,789,879,789->110->879
3,666,879,666->110->879
4,110,879,110->879


In [ ]:
import random

# Generate a synthetic dataset
def generate_synthetic_data(num_nodes):
    edges = []
    for i in range(num_nodes - 1):
        # Each node will have a directed edge to another random node to ensure connectivity
        # This ensures there is at least one path from any node to any other node
        next_node = random.randint(i + 1, num_nodes - 1)
        edges.append((i, next_node))
    return edges

# Number of nodes
NUM_NODES = 50000

# Generate edges
edges = generate_synthetic_data(NUM_NODES)

# Create a directed graph using the edges
G_synthetic = nx.DiGraph(edges)

# Sample: let's take the first 10 nodes and find the longest path for each
sample_nodes = list(range(10))
longest_paths_synthetic = {}
for start in sample_nodes:
    paths = list(iddfs(G_synthetic, start, MAX_DEPTH))
    if paths:
        longest_paths_synthetic[start] = max(paths, key=len)

# Convert the results to a DataFrame
results_synthetic = []
for start, path in longest_paths_synthetic.items():
    results_synthetic.append([path[0], path[-1], '->'.join(map(str, path))])

df_results_synthetic = pd.DataFrame(results_synthetic, columns=['Start Node', 'End Node', 'Path Length'])
df_results_synthetic['Path Length'] = df_results_synthetic['Path Length'].apply(len)  # Display only the length for simplicity
df_results_synthetic


,Start Node,End Node,Path Length
0,0,49999,71
1,1,49999,113
2,2,49999,99
3,3,49999,64
4,4,49999,71
5,5,49999,99
6,6,49999,110
7,7,49999,91
8,8,49999,106
9,9,49999,99


In [ ]:
# Convert the results to display the full paths
results_synthetic_full = []
for start, path in longest_paths_synthetic.items():
    results_synthetic_full.append([path[0], path[-1], '->'.join(map(str, path))])

df_results_synthetic_full = pd.DataFrame(results_synthetic_full, columns=['Start Node', 'End Node', 'Path'])
df_results_synthetic_full


,Start Node,End Node,Path
0,0,49999,0->48333->48421->49902->49938->49983->49991->4...
1,1,49999,1->30724->39500->40452->44004->46415->48594->4...
2,2,49999,2->41105->43365->46027->46643->46856->47136->4...
3,3,49999,3->18388->45418->49164->49805->49991->49993->4...
4,4,49999,4->35928->37352->40988->45127->45720->49883->4...
5,5,49999,5->21818->26596->30512->42227->47655->48136->4...
6,6,49999,6->428->6437->29834->33352->33640->38119->4439...
7,7,49999,7->5865->11643->27965->34932->42369->45987->47...
8,8,49999,8->33491->45237->49147->49507->49708->49786->4...
9,9,49999,9->28720->40716->42529->46524->46530->48819->4...


In [ ]:
# Find the longest path for every node using IDDFS
longest_paths_full_synthetic = {}
for start in range(NUM_NODES):
    paths = list(iddfs(G_synthetic, start, MAX_DEPTH))
    if paths:
        longest_paths_full_synthetic[start] = max(paths, key=len)

# Convert the results to a DataFrame
results_full_synthetic = []
for start, path in longest_paths_full_synthetic.items():
    results_full_synthetic.append([path[0], path[-1], '->'.join(map(str, path))])

df_results_full_synthetic = pd.DataFrame(results_full_synthetic, columns=['Start Node', 'End Node', 'Path'])

# Display the first few rows for clarity
df_results_full_synthetic.head()


In [ ]:
import pandas as pd
import numpy as np

# Initial dataframe
df = pd.DataFrame({
    'sending application': [123, 456, 789, 666, 110],
    'upstream application': [456, 789, 110, 110, 879]
})

# Generate synthetic data to reach approximately 50,000 nodes
# The approach is to replicate the transitions multiple times
num_replications = 50000 // len(df)
df_synthetic = pd.concat([df] * num_replications, ignore_index=True)

# Randomly shuffle the dataframe rows to make the dataset less predictable
df_synthetic = df_synthetic.sample(frac=1).reset_index(drop=True)

df_synthetic.head()


,sending application,upstream application
0,110,879
1,789,110
2,789,110
3,789,110
4,789,110


In [ ]:
df_synthetic

,sending application,upstream application
0,110,879
1,789,110
2,789,110
3,789,110
4,789,110
...,...,...
49995,110,879
49996,789,110
49997,123,456
49998,110,879


In [ ]:
import networkx as nx

# Create a directed graph
G = nx.from_pandas_edgelist(df_synthetic, 'sending application', 'upstream application', create_using=nx.DiGraph())

def find_all_paths(graph, source, target, path=[]):
    """
    Find all paths between a source and target node in a directed graph.
    """
    path = path + [source]
    if source == target:
        return [path]
    if source not in graph:
        return []
    paths = []
    for node in graph[source]:
        if node not in path:
            newpaths = find_all_paths(graph, node, target, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

# Find longest paths for each node
longest_paths = []
nodes = list(G.nodes())

for start_node in nodes:
    for end_node in nodes:
        if start_node != end_node:
            all_paths = find_all_paths(G, start_node, end_node)
            if all_paths:
                # Find the longest path among all paths
                longest = max(all_paths, key=len)
                longest_paths.append((start_node, end_node, longest))

# Convert results to a DataFrame
df_paths = pd.DataFrame(longest_paths, columns=['Start Node', 'End Node', 'Path'])

df_paths

,Start Node,End Node,Path
0,110,879,"[110, 879]"
1,789,110,"[789, 110]"
2,789,879,"[789, 110, 879]"
3,666,110,"[666, 110]"
4,666,879,"[666, 110, 879]"
5,123,110,"[123, 456, 789, 110]"
6,123,879,"[123, 456, 789, 110, 879]"
7,123,789,"[123, 456, 789]"
8,123,456,"[123, 456]"
9,456,110,"[456, 789, 110]"


In [ ]:
# Generate 50,000 unique nodes
nodes = np.arange(1, 50001)

# Randomly pair these nodes to create edges while ensuring multiple paths
np.random.shuffle(nodes)
num_edges = len(nodes) - 1  # For simplicity, we'll create n-1 edges

# Create sender and receiver (upstream) lists
senders = nodes[:num_edges]
receivers = nodes[1:num_edges+1]

# Construct the dataset
random_df = pd.DataFrame({
    'sending application': senders,
    'upstream application': receivers
})

# Shuffle the dataset to ensure randomness
random_df = random_df.sample(frac=1).reset_index(drop=True)

random_df

,sending application,upstream application
0,49289,42753
1,32137,36450
2,37919,17190
3,44778,32307
4,21988,26278
...,...,...
49994,19833,42109
49995,34611,26067
49996,26204,31939
49997,38128,32107


In [ ]:
def find_longest_path_BFS(graph, source, target):
    """
    Find the longest path between a source and target node using BFS.
    """
    visited = set()
    queue = [(source, [source])]
    longest_path = []

    while queue:
        (vertex, path) = queue.pop(0)
        if vertex not in visited:
            visited.add(vertex)
            for neighbor in graph[vertex]:
                if neighbor == target:
                    if len(path + [neighbor]) > len(longest_path):
                        longest_path = path + [neighbor]
                        print(longest_path)
                else:
                    queue.append((neighbor, path + [neighbor]))

    return longest_path

# Find longest paths for each node in the new random graph using BFS
longest_paths_BFS = []

# Due to the large size, we'll limit the number of nodes for which we calculate paths to avoid excessive computation
limited_nodes = nodes_random[:100]

for start_node in limited_nodes:
    for end_node in limited_nodes:
        if start_node != end_node:
            path = find_longest_path_BFS(G_random, start_node, end_node)
            if path:
                longest_paths_BFS.append((start_node, end_node, path))

# Convert results to a DataFrame
df_paths_BFS = pd.DataFrame(longest_paths_BFS, columns=['Start Node', 'End Node', 'Path'])

# Display the top few rows
df_paths_BFS

[49289, 42753]
[49289, 42753, 45420, 24561, 20089, 49544, 4424, 47599, 29584, 26019, 19580, 5886, 21319, 20033, 6443, 40659, 37142, 2259, 28457, 1216, 42502, 31328, 19979, 7833, 25427, 34662, 28941, 27485, 27561, 17449, 42714, 40822, 33111, 41517, 25621, 35015, 7075, 26048, 29804, 41494, 15167, 39415, 13046, 11491, 41501, 27278, 33353, 14011, 39282, 48791, 2801, 26333, 49369, 5967, 27764, 29698, 8545, 2403, 26106, 23241, 27562, 16162, 958, 46583, 32894, 39082, 1745, 42875, 38654, 14600, 4187, 45289, 41874, 44960, 20025, 6149, 28565, 27027, 13532, 34334, 42293, 15541, 35699, 39846, 48216, 16659, 21162, 43538, 23495, 30977, 26819, 39658, 5411, 34882, 7928, 34291, 28579, 36899, 7554, 30934, 29542, 35189, 19687, 40286, 34990, 43656, 6, 44513, 12920, 20844, 11069, 18436, 32609, 23815, 16885, 23245, 38267, 40262, 28982, 15214, 15494, 14353, 17318, 18289, 31365, 42755, 4771, 13320, 25591, 21961, 9575, 12602, 35425, 49751, 36375, 21410, 29104, 24340, 25752, 45418, 34739, 11286, 48953, 32319, 4

KeyboardInterrupt: ignored

In [14]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import deque

# Generating the random dataset and graph as done previously
nodes = np.arange(1, 50001)
np.random.shuffle(nodes)
num_edges = len(nodes) - 1
senders = nodes[:num_edges]
receivers = nodes[1:num_edges+1]
random_df = pd.DataFrame({
    'sending application': senders,
    'upstream application': receivers
})
G_random = nx.from_pandas_edgelist(random_df, 'sending application', 'upstream application', create_using=nx.DiGraph())

# Function for BFS path finding
def bfs_paths(G, start, is_goal, max_depth=None):
    visited = set([start])
    queue = deque([(start, [start])])
    while queue:
        current_node, path = queue.popleft()
        if max_depth and len(path) > max_depth:
            continue
        for neighbor in G.neighbors(current_node):
            if neighbor in visited:
                continue
            visited.add(neighbor)
            new_path = path + [neighbor]
            if is_goal(neighbor):

                yield new_path
            else:
                queue.append((neighbor, new_path))

# Compute the degree centrality for each node in the graph
degree_centrality = nx.degree_centrality(G_random)
print(degree_centrality)
# Sort nodes by their degree centrality and select the top 100
central_nodes = sorted(degree_centrality, key=degree_centrality.get, reverse=True)

# Find longest paths among the central nodes using the optimized BFS
longest_paths_central = []

for start_node in central_nodes:
    for end_node in central_nodes:
        if start_node != end_node:
            paths_gen = bfs_paths(G_random, start_node, lambda x: x == end_node)
            longest_path = max(paths_gen, key=len, default=[])
            if longest_path:
                longest_paths_central.append((start_node, end_node, longest_path))

# Convert results to a DataFrame
df_paths_central = pd.DataFrame(longest_paths_central, columns=['Start Node', 'End Node', 'Path'])

# Display the top few rows
df_paths_central.head()


{27760: 2.000040000800016e-05, 33639: 4.000080001600032e-05, 31163: 4.000080001600032e-05, 23451: 4.000080001600032e-05, 6002: 4.000080001600032e-05, 13978: 4.000080001600032e-05, 9703: 4.000080001600032e-05, 35639: 4.000080001600032e-05, 9587: 4.000080001600032e-05, 3137: 4.000080001600032e-05, 44368: 4.000080001600032e-05, 35346: 4.000080001600032e-05, 2410: 4.000080001600032e-05, 45914: 4.000080001600032e-05, 5353: 4.000080001600032e-05, 14307: 4.000080001600032e-05, 31568: 4.000080001600032e-05, 33669: 4.000080001600032e-05, 736: 4.000080001600032e-05, 19101: 4.000080001600032e-05, 36365: 4.000080001600032e-05, 48200: 4.000080001600032e-05, 28070: 4.000080001600032e-05, 48122: 4.000080001600032e-05, 1940: 4.000080001600032e-05, 46002: 4.000080001600032e-05, 20639: 4.000080001600032e-05, 35129: 4.000080001600032e-05, 49554: 4.000080001600032e-05, 29043: 4.000080001600032e-05, 16236: 4.000080001600032e-05, 13626: 4.000080001600032e-05, 40439: 4.000080001600032e-05, 27166: 4.000080001

KeyboardInterrupt: ignored

In [17]:
# Set a max depth for the BFS traversal
MAX_DEPTH = 5

# Find longest paths among the central nodes using the optimized BFS with a max depth
longest_paths_depth_limited = []

for start_node in central_nodes:
    for end_node in central_nodes:
        if start_node != end_node:
            paths_gen = bfs_paths(G_random, start_node, lambda x: x == end_node, max_depth=MAX_DEPTH)
            longest_path = max(paths_gen, key=len, default=[])
            if longest_path:
                longest_paths_depth_limited.append((start_node, end_node, longest_path))
                # print(longest_paths_depth_limited)

# Convert results to a DataFrame
df_paths_depth_limited = pd.DataFrame(longest_paths_depth_limited, columns=['Start Node', 'End Node', 'Path'])

# Display the top few rows
df_paths_depth_limited.head()


KeyboardInterrupt: ignored

In [ ]:
df_paths_depth_limited

In [19]:
# Create the DataFrame
data = {
    'sending application': [123, 456, 789, 666, 110],
    'upstream application': [456, 789, 110, 110, 879]
}
df = pd.DataFrame(data)

# Create the DiGraph from the DataFrame
G_random = nx.from_pandas_edgelist(df, 'sending application', 'upstream application', create_using=nx.DiGraph())


In [20]:
# Iteratively find the longest paths by increasing the BFS depth
depth = 2
previous_max_length = 0
all_longest_paths = []

while True:
    longest_paths_current_depth = []

    for start_node in central_nodes:
        for end_node in central_nodes:
            if start_node != end_node:
                paths_gen = bfs_paths(G_random, start_node, lambda x: x == end_node, max_depth=depth)
                longest_path = max(paths_gen, key=len, default=[])
                if longest_path:
                    longest_paths_current_depth.append((start_node, end_node, longest_path))

    # Check if the longest path's length has increased
    current_max_length = max([len(path[2]) for path in longest_paths_current_depth], default=0)

    if current_max_length > previous_max_length:
        all_longest_paths.extend(longest_paths_current_depth)
        previous_max_length = current_max_length
        depth += 1
    else:
        break

# Convert results to a DataFrame
df_all_longest_paths = pd.DataFrame(all_longest_paths, columns=['Start Node', 'End Node', 'Path'])

# Display the top few rows
df_all_longest_paths.head()


NetworkXError: ignored

In [22]:
# Creating a graph from the provided dataframe
data = {
    'sending application': [123, 456, 789, 666, 110],
    'upstream application': [456, 789, 110, 110, 879]
}

test_df = pd.DataFrame(data)
G_test = nx.from_pandas_edgelist(test_df, 'sending application', 'upstream application', create_using=nx.MultiDiGraph())

# Iteratively find the longest paths by increasing the BFS depth for the test graph
depth = 2
previous_max_length = 0
test_longest_paths = []
test_nodes = list(G_test.nodes())

while True:
    longest_paths_current_depth = []

    for start_node in test_nodes:
        for end_node in test_nodes:
            if start_node != end_node:
                paths_gen = bfs_paths(G_test, start_node, lambda x: x == end_node, max_depth=depth)
                longest_path = max(paths_gen, key=len, default=[])
                if longest_path:
                    longest_paths_current_depth.append((start_node, end_node, longest_path))

    # Check if the longest path's length has increased
    current_max_length = max([len(path[2]) for path in longest_paths_current_depth], default=0)

    if current_max_length > previous_max_length:
        test_longest_paths.extend(longest_paths_current_depth)
        previous_max_length = current_max_length
        depth += 1
    else:
        break

# Convert results to a DataFrame
df_test_longest_paths = pd.DataFrame(test_longest_paths, columns=['Start Node', 'End Node', 'Path'])

df_test_longest_paths


,Start Node,End Node,Path
0,123,456,"[123, 456]"
1,123,789,"[123, 456, 789]"
2,456,789,"[456, 789]"
3,456,110,"[456, 789, 110]"
4,789,110,"[789, 110]"
5,789,879,"[789, 110, 879]"
6,110,879,"[110, 879]"
7,666,110,"[666, 110]"
8,666,879,"[666, 110, 879]"
9,123,456,"[123, 456]"


In [23]:
# Generate 10,000 unique nodes
nodes_synthetic = np.arange(1, 10001)

# Randomly pair these nodes to create edges
np.random.shuffle(nodes_synthetic)
num_edges_synthetic = len(nodes_synthetic) - 1

# Create sender and receiver (upstream) lists
senders_synthetic = nodes_synthetic[:num_edges_synthetic]
receivers_synthetic = nodes_synthetic[1:num_edges_synthetic+1]

# Construct the synthetic dataset
synthetic_df = pd.DataFrame({
    'sending application': senders_synthetic,
    'upstream application': receivers_synthetic
})

# Shuffle the dataset to ensure randomness
synthetic_df = synthetic_df.sample(frac=1).reset_index(drop=True)

synthetic_df.head()


,sending application,upstream application
0,3429,7428
1,1484,49
2,9559,5745
3,9512,2905
4,9429,7729


In [ ]:
# Create a directed graph from the synthetic dataset
G_synthetic = nx.from_pandas_edgelist(synthetic_df, 'sending application', 'upstream application', create_using=nx.DiGraph())

# Iteratively find the longest paths by increasing the BFS depth for the synthetic graph
depth = 2
previous_max_length = 0
synthetic_longest_paths = []
synthetic_nodes = list(G_synthetic.nodes())

while True:
    longest_paths_current_depth = []

    for start_node in synthetic_nodes:
        for end_node in synthetic_nodes:
            if start_node != end_node:
                paths_gen = bfs_paths(G_synthetic, start_node, lambda x: x == end_node, max_depth=depth)
                longest_path = max(paths_gen, key=len, default=[])
                if longest_path:
                    longest_paths_current_depth.append((start_node, end_node, longest_path))

    # Check if the longest path's length has increased
    current_max_length = max([len(path[2]) for path in longest_paths_current_depth], default=0)

    if current_max_length > previous_max_length:
        synthetic_longest_paths.extend(longest_paths_current_depth)
        previous_max_length = current_max_length
        depth += 1
    else:
        break

# Convert results to a DataFrame
df_synthetic_longest_paths = pd.DataFrame(synthetic_longest_paths, columns=['Start Node', 'End Node', 'Path'])

# Display the top few rows
df_synthetic_longest_paths.head()


## final

In [1]:
import pandas as pd
import networkx as nx
from collections import deque

# Reconstructing the graph from the provided dataframe
data = {
    'sending application': [123, 456, 789, 666, 110],
    'upstream application': [456, 789, 110, 110, 879]
}

test_df = pd.DataFrame(data)
G_test = nx.from_pandas_edgelist(test_df, 'sending application', 'upstream application', create_using=nx.DiGraph())
test_nodes = list(G_test.nodes())

# Function for BFS path finding
def bfs_paths(G, start, is_goal, max_depth=None):
    visited = set([start])
    queue = deque([(start, [start])])
    while queue:
        current_node, path = queue.popleft()
        if max_depth and len(path) > max_depth:
            continue
        for neighbor in G.neighbors(current_node):
            if neighbor in visited:
                continue
            visited.add(neighbor)
            new_path = path + [neighbor]
            if is_goal(neighbor):
                yield new_path
            else:
                queue.append((neighbor, new_path))

# Finding the longest paths for the fixed starting nodes
fixed_longest_paths = []

for start_node in fixed_start_nodes:
    for end_node in test_nodes:
        if start_node != end_node:
            paths_gen = bfs_paths(G_test, start_node, lambda x: x == end_node)
            longest_path = max(paths_gen, key=len, default=[])
            if longest_path:
                fixed_longest_paths.append((start_node, end_node, longest_path))

# Convert results to a DataFrame
df_fixed_longest_paths = pd.DataFrame(fixed_longest_paths, columns=['Start Node', 'End Node', 'Path'])

df_fixed_longest_paths


NameError: ignored

In [2]:
# Define the fixed starting nodes
fixed_start_nodes = [123, 666]

# Finding the longest paths for the fixed starting nodes
fixed_longest_paths = []

for start_node in fixed_start_nodes:
    for end_node in test_nodes:
        if start_node != end_node:
            paths_gen = bfs_paths(G_test, start_node, lambda x: x == end_node)
            longest_path = max(paths_gen, key=len, default=[])
            if longest_path:
                fixed_longest_paths.append((start_node, end_node, longest_path))

# Convert results to a DataFrame
df_fixed_longest_paths = pd.DataFrame(fixed_longest_paths, columns=['Start Node', 'End Node', 'Path'])

df_fixed_longest_paths


,Start Node,End Node,Path
0,123,456,"[123, 456]"
1,123,789,"[123, 456, 789]"
2,123,110,"[123, 456, 789, 110]"
3,123,879,"[123, 456, 789, 110, 879]"
4,666,110,"[666, 110]"
5,666,879,"[666, 110, 879]"
